In [1]:
import numpy as np
import pandas as pd
from puzzle import Puzzle

In [3]:
a = Puzzle()

In [10]:
a.verify_solution(['2', 'L'],[[{'4', '6', '6 Express'}, {'L'}]])

False

In [10]:
a.subway_table.sample(2)['NAME'].values

array(['Beach 67th St', 'Broadway Junction'], dtype=object)

In [12]:
import json
json.dumps([[[1,2],[3,4]]])

'[[[1, 2], [3, 4]]]'

In [13]:
json.loads('')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
def get_coords_from_string(string):
    return np.array(string.split('(')[1].split(')')[0].split(' ')).astype(float)

In [3]:
a = pd.read_csv('./data/subway.csv').drop(columns=['URL','OBJECTID'])
a['the_geom'] = [get_coords_from_string(x) for x in a['the_geom']]
a['LINE'] = [x.split('-') for x in a['LINE'].values]
a['NOTES'] = [x.split(',') for x in a['NOTES'].values]


In [4]:
a.head()

,NAME,the_geom,LINE,NOTES
0,Astor Pl,"[-73.99106999861966, 40.73005400028978]","[4, 6, 6 Express]","[4 nights, 6-all times, 6 Express-weekdays A..."
1,Canal St,"[-74.00019299927328, 40.71880300107709]","[4, 6, 6 Express]","[4 nights, 6-all times, 6 Express-weekdays A..."
2,50th St,"[-73.98384899986625, 40.76172799961419]","[1, 2]","[1-all times, 2-nights]"
3,Bergen St,"[-73.97499915116808, 40.68086213682956]","[2, 3, 4]","[4-nights, 3-all other times, 2-all times]"
4,Pennsylvania Ave,"[-73.89488591154061, 40.66471445143568]","[3, 4]","[4-nights, 3-all other times]"


In [5]:
def get_coord_dist(coords1, coords2):
    return np.sqrt(np.sum((np.array(coords1) - np.array(coords2))**2))

In [6]:
THRESHOLD = 0.002
indices_to_drop = []
for i in range(len(a)):
    if i in indices_to_drop:
        continue
    try:
        current_station = a.loc[i]
    except KeyError:
        continue
    for j in range(len(a)):
        if j == i or j in indices_to_drop:
            continue
        else:
            try:
                candidate_station = a.loc[j]
            except KeyError:
                continue
            if current_station['NAME'] == candidate_station['NAME']:
                if get_coord_dist(current_station['the_geom'],candidate_station['the_geom']) < THRESHOLD:
                    [current_station['LINE'].append(x) for x in candidate_station['LINE']]
                    [current_station['NOTES'].append(x) for x in candidate_station['NOTES']]
                    current_station['LINE'] = list(set(current_station['LINE']))
                    current_station['NOTES'] = list(set(current_station['NOTES']))
                    indices_to_drop.append(j)
                else:
                    candidate_station['NAME'] = candidate_station['NAME'] + '_'
df = a.drop(indices_to_drop)
df = df.reset_index().drop('index',axis=1)

In [21]:
df[df['NAME']=='Metropolitan Ave']

,NAME,the_geom,LINE,NOTES
144,Metropolitan Ave,"[-73.9514239994525, 40.71277400073426]",[G],[G-all times]


In [8]:
station_graph = {}
for i in range(len(df)):
    station = df.loc[i]['NAME']
    # if station in list(station_graph.keys()):
    #     station = station + '_'
    connections = []
    for j in range(len(df)):
        if j == i:
            continue
        else:
            a_set = set(df.loc[i]['LINE'])
            b_set = set(df.loc[j]['LINE'])
            if len(a_set.intersection(b_set)) > 0:
                connections.append(df.loc[j]['NAME'])
    station_graph[station] = connections


In [97]:
def find_all_paths(graph, start, end, path=[]):
    if start in path:
        return []
    path = path + [start]
    if start == end:
        return [path]
    if not start in list(graph.keys()):
        return []
    paths = []
    for node in graph[start]:
        if node not in path and len(path) < 4:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths
def find_shortest_paths(graph, start, end, path=[]):
    paths = find_all_paths(station_graph, start, end)
    return [x for x in paths if len(x) == len(min(paths, key=len))]
def find_common_lines(station1, station2):
    return set(list(a[a['NAME'] == station1]['LINE'].values)[0]).intersection(set(list(a[a['NAME'] == station2]['LINE'].values)[0]))
def find_connecting_lines(paths, unique = True):
    line_paths = []
    for path in paths:
        lines = []
        for i in range(len(path)-1):
            lines.append(find_common_lines(path[i],path[i+1]))
        if lines not in line_paths or not unique:
            line_paths.append(lines)
    return line_paths
def verify_solution(answer, solution):
    for y in range(np.shape(solution)[1]):
        if y == 0:
            verify = np.array([answer[y] in x for x in np.array(solution)[:,y].flatten()])
        else:
            verify = verify & np.array([answer[y] in x for x in np.array(solution)[:,y].flatten()])
    return bool(verify)

In [98]:
find_shortest_paths(station_graph, 'Astor Pl', '50th St')

[['Astor Pl', 'Bergen St', '50th St'],
 ['Astor Pl', 'Franklin Ave', '50th St'],
 ['Astor Pl', '149th St - Grand Concourse', '50th St'],
 ['Astor Pl', 'Grand Army Plaza', '50th St'],
 ['Astor Pl', "Atlantic Av - Barclay's Center", '50th St'],
 ['Astor Pl', 'Borough Hall', '50th St'],
 ['Astor Pl', 'Nevins St', '50th St'],
 ['Astor Pl', 'Eastern Pkwy - Bklyn Museum', '50th St'],
 ['Astor Pl', 'Fulton St_', '50th St']]

In [99]:
test = find_connecting_lines(find_shortest_paths(station_graph, 'Astor Pl', 'Hunters Point Ave'))
# fewest_subways = np.array(test).shape[1]
# num_solutions = np.array(test).shape[0]


In [101]:
verify_solution(['6 Express','7'], test)

True